In [1]:
import pandas as pd
import numpy as np
import json
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import time

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
from sklearn.model_selection import KFold

In [2]:
train = pd.read_csv('../../data/raw/gamma-log-facies/CAX_LogFacies_Train_File.csv')
print(train.shape)
train.head()

(4400000, 4)


,row_id,well_id,GR,label
0,0,0,143.510000,0
1,1,0,112.790928,0
2,2,0,123.531856,0
3,3,0,111.692784,0
4,4,0,123.613712,0


In [3]:
X_train = pd.pivot_table(train, values='GR', index=['well_id'], columns=['row_id'])
y_train = pd.pivot_table(train, values='label', index=['well_id'], columns=['row_id'])
X_train.shape, y_train.shape

((4000, 1100), (4000, 1100))

In [4]:
X_train.head()

row_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,...,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099

In [5]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, checkpoint_path='checkpoint.pt', is_maximize=True):
        self.patience, self.delta, self.checkpoint_path = patience, delta, checkpoint_path
        self.counter, self.best_score = 0, None
        self.is_maximize = is_maximize

    def load_best_weights(self, model):
        model.load_state_dict(torch.load(self.checkpoint_path))

    def __call__(self, score, model):
        if self.best_score is None or \
        (score > self.best_score + self.delta if self.is_maximize else score < self.best_score - self.delta):
            torch.save(model.state_dict(), self.checkpoint_path)
            self.best_score, self.counter = score, 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [6]:
class TabularDataset(Dataset):

    def __init__(self, df_x, df_y, is_test=False):
        self.x = df_x
        self.y = df_y
        self.n = df_x.shape[0]
        self.is_test=is_test

    def __len__(self): return self.n

    def __getitem__(self, idx): 
        if not self.is_test:
            return [self.x[idx].astype(np.float32), self.y[idx].astype(np.int64)]
        else:
            return [self.x[idx].astype(np.float32), self.y[idx]]

In [7]:
class Seq2SeqRnn(nn.Module):
    def __init__(self, input_size, seq_len, hidden_size, output_size, num_layers=1, bidirectional=False, dropout=.3,
            hidden_layers = [100, 200]):
        
        super().__init__()
        self.input_size = input_size
        self.seq_len = seq_len
        self.hidden_size = hidden_size
        self.num_layers=num_layers
        self.bidirectional=bidirectional
        self.output_size=output_size
        
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, 
                           bidirectional=bidirectional, batch_first=True,dropout=.3)
        
         # Input Layer
        if hidden_layers and len(hidden_layers):
            first_layer  = nn.Linear(hidden_size*2 if bidirectional else hidden_size, hidden_layers[0])

            # Hidden Layers
            self.hidden_layers = nn.ModuleList(
                [first_layer]+[nn.Linear(hidden_layers[i], hidden_layers[i+1]) for i in range(len(hidden_layers) - 1)]
            )
            for layer in self.hidden_layers: nn.init.kaiming_normal_(layer.weight.data)   

            # output layers
            self.output_layer = nn.Linear(hidden_layers[-1], output_size)
            nn.init.kaiming_normal_(self.output_layer.weight.data) 
           
        else:
            self.hidden_layers = []
            self.output_layer = nn.Linear(hidden_size*2 if bidirectional else hidden_size, output_size)
            nn.init.kaiming_normal_(self.output_layer.weight.data) 
            
        self.bn_layers = nn.ModuleList([nn.BatchNorm1d(seq_len) for size in hidden_layers])
        self.activation_fn = torch.relu
            
        self.dropout = nn.Dropout(dropout)
        self.output_activation_fn = partial(torch.softmax, dim=1)
        
    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, self.seq_len, self.input_size) 
        outputs, (hidden, cell) = self.rnn(x)        

        x = self.dropout(self.activation_fn(outputs))
        for hidden_layer, bn_layer in zip(self.hidden_layers, self.bn_layers):
            x = self.activation_fn(hidden_layer(x))
#             x = bn_layer(x)
            x = self.dropout(x)
        
        x = self.output_layer(x)
#         x = self.output_activation_fn(x)
        return x

In [8]:
folds = KFold(n_splits=5, random_state=100, shuffle=True)
indices = [(train_index, test_index) for (train_index, test_index) in folds.split(X_train.index)]
train_index, val_index = indices[2]
len(train_index), len(val_index)

(3200, 800)

In [9]:
for index, (train_index, val_index ) in enumerate(indices):
    print("Fold : {}".format(index))
    
    batchsize = 16
    train_dataset = TabularDataset(df_x=X_train.iloc[train_index].values,  df_y=y_train.iloc[train_index].values)
    train_dataloader = DataLoader(train_dataset, batchsize, shuffle=True, num_workers=1)

    valid_dataset = TabularDataset(df_x=X_train.iloc[val_index].values,  df_y=y_train.iloc[val_index].values)
    valid_dataloader = DataLoader(valid_dataset, batchsize, shuffle=True, num_workers=1)

    # test_dataset = TabularDataset(df_x=X_test.values, df_y=X_test_ids.values, is_test=True)
    # test_dataloader = DataLoader(test_dataset, batchsize, shuffle=False, num_workers=1)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model = Seq2SeqRnn(input_size=1, seq_len=1100, hidden_size=256, output_size=5, num_layers=3, hidden_layers=[1024],
                     bidirectional=True).to(device)

    no_of_epochs = 100
    early_stopping = EarlyStopping(patience=10, is_maximize=True, checkpoint_path="checkpoint_{}.pt".format(index))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    schedular = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=0.001, epochs=no_of_epochs,
                                            steps_per_epoch=len(train_dataloader))
    avg_train_losses, avg_valid_losses = [], [] 


    for epoch in range(no_of_epochs):
        start_time = time.time()

        print("Epoch : {}".format(epoch))

        train_losses, valid_losses = [], []

        model.train() # prep model for training
        train_preds, train_true = torch.Tensor([]).to(device), torch.LongTensor([]).to(device)

        for x, y in train_dataloader:          
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            predictions = model(x)

            predictions_ = predictions.view(-1, predictions.shape[-1]) 
            y_ = y.view(-1)

            loss = criterion(predictions_, y_)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            schedular.step()
            # record training loss
            train_losses.append(loss.item())

            train_true = torch.cat([train_true, y_], 0)
            train_preds = torch.cat([train_preds, predictions_], 0)



        model.eval() # prep model for evaluation
        val_preds, val_true = torch.Tensor([]).to(device), torch.LongTensor([]).to(device)
        for x, y in valid_dataloader:
            x = x.to(device)
            y = y.to(device)

            predictions = model(x)
            predictions_ = predictions.view(-1, predictions.shape[-1]) 
            y_ = y.view(-1)

            loss = criterion(predictions_, y_)
            valid_losses.append(loss.item())

            val_true = torch.cat([val_true, y_], 0)
            val_preds = torch.cat([val_preds, predictions_], 0)


        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        print( "train_loss: {}, valid_loss: {}".format(train_loss, valid_loss))

        train_score = accuracy_score(train_preds.cpu().detach().numpy().argmax(1), train_true.cpu().detach().numpy())

        val_score = accuracy_score(val_preds.cpu().detach().numpy().argmax(1), val_true.cpu().detach().numpy())
        print( "train_acc: {}, valid_acc: {}".format(train_score, val_score))

        if early_stopping(val_score, model):
            print("Early Stopping...")
            print("Best Val Score: {}".format(early_stopping.best_score))
            break

        print("--- %s seconds ---" % (time.time() - start_time))

Fold : 0
Epoch : 0


RuntimeError: CUDA out of memory. Tried to allocate 154.00 MiB (GPU 0; 7.93 GiB total capacity; 5.44 GiB already allocated; 159.50 MiB free; 5.90 GiB reserved in total by PyTorch)

In [ ]:
for index, (train_index, val_index ) in enumerate(indices):
    print("Fold : {}".format(index))
    
    batchsize = 16
    train_dataset = TabularDataset(df_x=X_train.iloc[train_index].values,  df_y=y_train.iloc[train_index].values)
    train_dataloader = DataLoader(train_dataset, batchsize, shuffle=True, num_workers=1)

    valid_dataset = TabularDataset(df_x=X_train.iloc[val_index].values,  df_y=y_train.iloc[val_index].values)
    valid_dataloader = DataLoader(valid_dataset, batchsize, shuffle=True, num_workers=1)

    # test_dataset = TabularDataset(df_x=X_test.values, df_y=X_test_ids.values, is_test=True)
    # test_dataloader = DataLoader(test_dataset, batchsize, shuffle=False, num_workers=1)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    model=Seq2SeqRnn(input_size=1, seq_len=1100, hidden_size=256, output_size=5, num_layers=3, hidden_layers=[1024],
                     bidirectional=True).to(device)

    no_of_epochs = 100
    early_stopping = EarlyStopping(patience=10, is_maximize=True, checkpoint_path="checkpoint_{}.pt".format(index))
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    schedular = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=0.001, epochs=no_of_epochs,
                                            steps_per_epoch=len(train_dataloader))
    avg_train_losses, avg_valid_losses = [], [] 


    for epoch in range(no_of_epochs):
        start_time = time.time()

        print("Epoch : {}".format(epoch))

        train_losses, valid_losses = [], []

        model.train() # prep model for training
        train_preds, train_true = torch.Tensor([]).to(device), torch.LongTensor([]).to(device)

        for x, y in train_dataloader:          
            x = x.to(device)
            y = y.to(device)

            optimizer.zero_grad()
            predictions = model(x)

            predictions_ = predictions.view(-1, predictions.shape[-1]) 
            y_ = y.view(-1)

            loss = criterion(predictions_, y_)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            schedular.step()
            # record training loss
            train_losses.append(loss.item())

            train_true = torch.cat([train_true, y_], 0)
            train_preds = torch.cat([train_preds, predictions_], 0)



        model.eval() # prep model for evaluation
        val_preds, val_true = torch.Tensor([]).to(device), torch.LongTensor([]).to(device)
        for x, y in valid_dataloader:
            x = x.to(device)
            y = y.to(device)

            predictions = model(x)
            predictions_ = predictions.view(-1, predictions.shape[-1]) 
            y_ = y.view(-1)

            loss = criterion(predictions_, y_)
            valid_losses.append(loss.item())

            val_true = torch.cat([val_true, y_], 0)
            val_preds = torch.cat([val_preds, predictions_], 0)


        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)

        print( "train_loss: {}, valid_loss: {}".format(train_loss, valid_loss))

        train_score = accuracy_score(train_preds.cpu().detach().numpy().argmax(1), train_true.cpu().detach().numpy())

        val_score = accuracy_score(val_preds.cpu().detach().numpy().argmax(1), val_true.cpu().detach().numpy())
        print( "train_acc: {}, valid_acc: {}".format(train_score, val_score))

        if early_stopping(val_score, model):
            print("Early Stopping...")
            print("Best Val Score: {}".format(early_stopping.best_score))
            break

        print("--- %s seconds ---" % (time.time() - start_time))